<a href="https://colab.research.google.com/github/aryaneelshivam/ArysStockAnalysis/blob/main/ArysStockAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Install required packages
# !pip install dash yfinance pandas

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

# Load stock data
stock_symbol = "BEL.NS"
start_date = "2022-01-01"
end_date = pd.to_datetime("today").strftime('%Y-%m-%d')
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

# Calculate buy/sell signals
stock_data['EMA5'] = stock_data['Close'].ewm(span=5).mean()
stock_data['EMA15'] = stock_data['Close'].ewm(span=15).mean()
stock_data['SMA5'] = stock_data['Close'].rolling(window=5).mean()
stock_data['SMA15'] = stock_data['Close'].rolling(window=15).mean()

def buy_sell(stock_data):
    # Function for buy and sell signal
    signalBuy = []
    signalSell = []
    position = False

    for i in range(len(stock_data)):
        if stock_data['SMA5'][i] > stock_data['SMA15'][i]:
            if position == False:
                signalBuy.append(stock_data['Adj Close'][i])
                signalSell.append(None)
                position = True
            else:
                signalBuy.append(None)
                signalSell.append(None)
        elif stock_data['SMA5'][i] < stock_data['SMA15'][i]:
            if position == True:
                signalBuy.append(None)
                signalSell.append(stock_data['Adj Close'][i])
                position = False
            else:
                signalBuy.append(None)
                signalSell.append(None)
        else:
            signalBuy.append(None)
            signalSell.append(None)
    return pd.Series([signalBuy, signalSell])

stock_data[['Buy_Signal_price', 'Sell_Signal_price']] = buy_sell(stock_data)

def buy_sellema(stock_data):
    signalBuyema = []
    signalSellema = []
    position = False

    for i in range(len(stock_data)):
        if stock_data['EMA5'][i] > stock_data['EMA15'][i]:
            if position == False:
                signalBuyema.append(stock_data['Adj Close'][i])
                signalSellema.append(None)
                position = True
            else:
                signalBuyema.append(None)
                signalSellema.append(None)
        elif stock_data['EMA5'][i] < stock_data['EMA15'][i]:
            if position == True:
                signalBuyema.append(None)
                signalSellema.append(stock_data['Adj Close'][i])
                position = False
            else:
                signalBuyema.append(None)
                signalSellema.append(None)
        else:
            signalBuyema.append(None)
            signalSellema.append(None)
    return pd.Series([signalBuyema, signalSellema])

stock_data[['Buy_Signal_priceEMA', 'Sell_Signal_priceEMA']] = buy_sellema(stock_data)

# Dash app
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children=f'{stock_symbol} Stock Price History with Buy/Sell Signals'),

    dcc.Graph(
        id='price-chart',
        figure=go.Figure(data=[
            go.Scatter(x=stock_data.index, y=stock_data['Adj Close'], mode='lines', name='Adj Close'),
            go.Scatter(x=stock_data.index, y=stock_data['SMA5'], mode='lines', name='SMA5'),
            go.Scatter(x=stock_data.index, y=stock_data['SMA15'], mode='lines', name='SMA15'),
            go.Scatter(x=stock_data.index, y=stock_data['EMA5'], mode='lines', name='EMA5'),
            go.Scatter(x=stock_data.index, y=stock_data['EMA15'], mode='lines', name='EMA15'),
            go.Scatter(x=stock_data.index, y=stock_data['Buy_Signal_price'], mode='markers', name='Buy SMA', marker=dict(color='green', size=8)),
            go.Scatter(x=stock_data.index, y=stock_data['Sell_Signal_price'], mode='markers', name='Sell SMA', marker=dict(color='red', size=8)),
            go.Scatter(x=stock_data.index, y=stock_data['Buy_Signal_priceEMA'], mode='markers', name='Buy EMA', marker=dict(color='black', size=8)),
            go.Scatter(x=stock_data.index, y=stock_data['Sell_Signal_priceEMA'], mode='markers', name='Sell EMA', marker=dict(color='purple', size=8))
        ]),
        style={'height': 500},
    ),

     dcc.Graph(
        id='price-chartSMA',
        figure=go.Figure(data=[
            go.Scatter(x=stock_data.index, y=stock_data['Adj Close'], mode='lines', name='Adj Close'),
            go.Scatter(x=stock_data.index, y=stock_data['SMA5'], mode='lines', name='SMA5'),
            go.Scatter(x=stock_data.index, y=stock_data['SMA15'], mode='lines', name='SMA15'),
            go.Scatter(x=stock_data.index, y=stock_data['Buy_Signal_price'], mode='markers', name='Buy SMA', marker=dict(color='green', size=8)),
            go.Scatter(x=stock_data.index, y=stock_data['Sell_Signal_price'], mode='markers', name='Sell SMA', marker=dict(color='red', size=8)),
        ]),
        style={'height': 500},
    ),

     dcc.Graph(
        id='price-chartEMA',
        figure=go.Figure(data=[
            go.Scatter(x=stock_data.index, y=stock_data['Adj Close'], mode='lines', name='Adj Close'),
            go.Scatter(x=stock_data.index, y=stock_data['EMA5'], mode='lines', name='EMA5'),
            go.Scatter(x=stock_data.index, y=stock_data['EMA15'], mode='lines', name='EMA15'),
            go.Scatter(x=stock_data.index, y=stock_data['Buy_Signal_priceEMA'], mode='markers', name='Buy EMA', marker=dict(color='black', size=8)),
            go.Scatter(x=stock_data.index, y=stock_data['Sell_Signal_priceEMA'], mode='markers', name='Sell EMA', marker=dict(color='purple', size=8))
        ]),
        style={'height': 500},
    ),

    dcc.Graph(
        id='volume-chart',
        figure=go.Figure(data=[
            go.Scatter(x=stock_data.index, y=stock_data['Volume'], mode='lines', name='Volume', line=dict(color='purple'))
        ]),
        style={'height': 300},
    ),
])

if __name__ == '__main__':
    app.run_server(debug=True)


[*********************100%%**********************]  1 of 1 completed


<IPython.core.display.Javascript object>